<a href="https://colab.research.google.com/github/pcpiscator/2T2021/blob/main/Furg_ECD_Machine_Learning_II_Semana_08_Vis%C3%A3o_computacional_usando_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning II - Visão computacional usando CNNs
### Prof. Marcelo Malheiros

Parte do código adaptada de Aurélien Geron (licença Apache-2.0)

---

# **Atenção**

Dada a complexidade das redes utilizadas, é recomendável executar este _notebook_ de forma que a biblioteca TensorFlow utilize uma GPU para seu processamento. Dentro do Colaboratory é preciso mudar o tipo de ambiente _runtime_ deste notebook. Então selecione a opção:

`Runtime > Change runtime type > Hardware accelerator: GPU`

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Pandas: pacote estatístico e de manipulação de DataFrames
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

Este _notebook_, em particular, utiliza a biblioteca Keras para definir e treinar redes neurais. Aqui utilizamos a versão **integrada** de Keras, que já vem como parte da biblioteca mais geral TensorFlow. Ambas já fazem parte do ambiente Colaboratory.

Para quem utiliza o ambiente Anaconda, é preciso primeiro instalar o pacote `tensorflow`. Isso pode ser feito com o seguinte comando:

    conda install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras

print('tensorflow:      versão', tf.__version__)
print('keras integrada: versão', keras.__version__)

Este _notebook_ também utiliza a biblioteca `pydot` e a ferramenta Graphviz para visualizar as redes neurais. Ambos já fazem parte do ambiente Colaboratory.

Para quem utiliza o ambiente Anaconda, é preciso primeiro instalar os pacotes `pydot` e `graphviz`. Isso pode ser feito com o seguinte comando:

    conda install pydot graphviz

In [ ]:
import pydot

# Redes neurais convolucionais

## Filtros

Aqui mostramos um exemplo simples da operação de convolução, definindo dois filtros simples e aplicando os mesmos a duas imagens.

In [ ]:
from sklearn.datasets import load_sample_image

# carrega duas imagens de exemplo da biblioteca Scikit-Learn
china = load_sample_image('china.jpg') / 255
flower = load_sample_image('flower.jpg') / 255

# as imagens precisam estar agrupadas em um conjunto
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

In [ ]:
# definição de dois filtros simples
filters = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)
filters[:, 3, :, 0] = 1  # linha vertical
filters[3, :, :, 1] = 1  # linha horizontal

In [ ]:
# operação de convolução sobre as duas imagens, usando os dois filtros
outputs = tf.nn.conv2d(images, filters, strides=1, padding='SAME')

In [ ]:
# funções auxiliares

def plot_image(image, axis='off'):
    plt.imshow(image, cmap='gray', interpolation='nearest')
    plt.axis(axis)

def plot_color_image(image, axis='off'):
    plt.imshow(image, interpolation='nearest')
    plt.axis(axis)

In [ ]:
# exibição dos resultados
plt.figure(figsize=(14, 4), tight_layout=True)
for image_index in (0, 1):
    for feature_map_index in (0, 1):
        plt.subplot(1, 4, image_index * 2 + feature_map_index + 1)
        plot_image(outputs[image_index, :, :, feature_map_index])
plt.show()

In [ ]:
# exibição de detalhes dos resultados

# função auxiliar
def crop(images):
    return images[150:220, 130:250]

plt.figure(figsize=(14, 4), tight_layout=True)
plt.subplot(1, 3, 1)
plot_image(crop(images[0, :, :, 0]))
image_index = 0
for feature_map_index in (0, 1):
    plt.subplot(1, 3, image_index * 2 + feature_map_index + 2)
    plot_image(crop(outputs[image_index, :, :, feature_map_index]))
plt.show()

In [ ]:
# exibição dos filtros como imagens
plt.figure(figsize=(6, 3), tight_layout=True)
plt.subplot(1, 2, 1)
plot_image(filters[:, :, 0, 0], 'on')
plt.subplot(1, 2, 2)
plot_image(filters[:, :, 0, 1], 'on')
plt.show()

## Camadas de convolução

Aqui vamos ilustrar a criação e funcionamento de uma camada de convolução bidimensional, especificada pela função `Conv2D` da biblioteca Keras.

Vamos criar uma camada com 2 filtros (`filters=2`), cada um sendo um quadrado de lado 7 (`kernel_size=7`). O espaçamento entre filtros é 1 _pixel_ (`strides=1`), ou seja, são aplicados a conjuntos contiguos de _pixels_.

O preenchimento das bordas com valores zero é dado pelo parâmetro `padding='SAME'`, enquanto a função de ativação a ser aplicada é especificada por `activation='relu'`.

Finalmente, também precisamos especificar o formato da entrada com `input_shape=outputs.shape`, que precisa ser compatível com dados a serem passados a esta camada.


In [ ]:
# inicialização das sementes aleatórias para repetibilidade
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# criação da camada
conv = keras.layers.Conv2D(filters=2, kernel_size=7, strides=1,
                           padding='SAME', activation='relu',
                           input_shape=images.shape)

In [ ]:
# aplicação para as duas imagens anteriores
conv_outputs = conv(images)
conv_outputs.shape 

A saída é uma matriz de quatro dimensões (chamada de **tensor** no contexto da biblioteca TensorFlow). As dimensões são: tamanho do lote, altura, largura e canais.

A primeira dimensão (tamanho do lote) é 2, pois há 2 imagens de entrada. As próximas duas dimensões são a altura e largura dos mapas de _features_ da saída: uma vez que `padding='SAME'` e `strides=1`, os mapas de _features_ de saída têm a mesma altura e largura das imagens de entrada (neste caso, 427 por 640 _pixels_). Por último, esta camada convolucional tem 2 filtros, então a última dimensão é 2: há 2 mapas de _features_ de saída por imagem de entrada.

Como os filtros são inicializados aleatoriamente, eles inicialmente detectam padrões aleatórios. Abaixo vamos exibir os mapas de _features_ de cada imagem:


In [ ]:
# exibição dos resultados
plt.figure(figsize=(14, 4), tight_layout=True)
for image_index in (0, 1):
    for feature_map_index in (0, 1):
        plt.subplot(1, 4, image_index * 2 + feature_map_index + 1)
        plot_image(crop(conv_outputs[image_index, :, :, feature_map_index]))
plt.show()

Embora os filtros tenham sido inicializados aleatoriamente, o segundo filtro age como um **detector de borda**.

Filtros inicializados aleatoriamente geralmente agem dessa maneira, o que é uma muito útil, pois detectar bordas é fundamental no processamento de imagens.

## Camadas de agrupamento

Aqui vamos demonstrar manualmente o funcionamento de dois tipos de camadas de agrupamento (_pooling layers_).

O tipo mais comum é o _max pooling_, em que apenas o maior elemento de um grupo de entradas é mantido.

Outro tipo, menos comum mas ainda assim útil em alguns contextos, é o _average pooling_. Como diz seu nome, ele retorna a média aritmética dos valores.

No dois casos, como a camada de agrupamento condensa _pixels_ de sua entrada, a saída é sempre uma imagem de resolução menor. Neste caso, com uma redução pela metade, dada pelo parâmetro `pool_size=2`.

In [ ]:
import matplotlib as mpl

# função auxiliar de visualização
def plot_max_avg(image, max_output, avg_output):
    fig = plt.figure(figsize=(12, 8))
    gs = mpl.gridspec.GridSpec(nrows=1, ncols=3, width_ratios=[2, 1, 1])
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.set_title('entrada', fontsize=14)
    ax1.imshow(image)
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.set_title('max', fontsize=14)
    ax2.imshow(max_output)
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.set_title('average', fontsize=14)
    ax3.imshow(avg_output)
    plt.show()

In [ ]:
# aplicação nos recortes das imagens
cropped_images = np.array([crop(image) for image in images], dtype=np.float32)

In [ ]:
# camada de max pooling
max_pool = keras.layers.MaxPool2D(pool_size=2)
max_output = max_pool(cropped_images)

In [ ]:
# camada de average pooling
avg_pool = keras.layers.AvgPool2D(pool_size=2)
avg_output = avg_pool(cropped_images)

In [ ]:
# visualização
plot_max_avg(cropped_images[0], max_output[0], avg_output[0])

# Classificador de imagens usando uma CNN

Aqui vamos construir uma classificador usando uma arquitetura tradicional de CNN para usar no _dataset_ **Fashion MNIST**, com imagens reduzidas de roupas.

Vamos fazer a abordagem usual de construir conjuntos de **treino**, **validação** e de **teste**, medindo ao final a acurácia da predição sobre o conjunto de teste.

## Conjunto de dados

A biblioteca Keras tem várias funções para carregar conjuntos de dados populares em `keras.datasets`. 

O _dataset_ **Fashion MNIST** já está dividido entre instâncias de treinamento e de teste, mas a seguir iremos dividir o conjunto de treinamento para ter um conjunto de validação. Cada instância é uma imagem em tons de cinza (com valores de 0 a 255) e com resolução 28 por 28 _pixels_.

Note que esse _dataset_ foi feito para ser compatível com o conjunto **MNIST** original, tendo a mesma resolução, número de instâncias e número de classes (10), porém sendo mais desafiador de classificar.

In [ ]:
# importação do dataset
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
print('treinamento completo:', X_train_full.shape)

Aqui o conjunto completo de treinamento é quebrado em dois, um de treinamento menor e outro de validação. Também é feita a conversão dos valores inteiros de tons de cinza (de 0 a 255) para um valor real no intervalo de 0 a 1.

In [ ]:
# separação dos dados de treinamento e validação
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

# adição de mais uma dimensão, para indicar o único canal de cor (tons de cinza)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
print('treinamento:', X_train.shape)
print('validação:   ', X_valid.shape)
print('testes:     ', X_test.shape)

Os rótulos são valores inteiros de 0 a 9, guardados nos vetores `y` e que correspondem aos seguintes nomes de classes.

In [ ]:
class_names = ['camiseta', 'calça', 'pulôver', 'vestido', 'casaco',
               'sandália', 'camisa', 'tênis', 'bolsa', 'bota']

Abaixo é exibido um mosaico com várias instâncias do conjunto de treino:

In [ ]:
# visualização das instâncias
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index, :, :, 0], cmap='binary')
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

## Criação da rede neural convolucional

In [ ]:
# comando para 'zerar' a biblioteca Keras
keras.backend.clear_session()

# definição de sementes aleatórias
np.random.seed(42)
tf.random.set_seed(42)

Aqui vamos criar uma rede neural de classificação, usando um modelo (ou arquitetura) do tipo sequencial. O modelo sequencial corresponde ao tipo mais simples de rede neural, onde uma sequência de camadas de neurônios é empilhada uma em cima da outra.

- A criação começa com a chamada a `Sequential`, que define o tipo do modelo:

        model = keras.models.Sequential()

- Então uma camada convolucional do tipo `Conv2D` é adicionada. Ela define 32 filtros, cada um com tamanho 3 por 3. O parâmetro _strides_ é 1 e o preenchimento das bordas é feito com zeros, então a camada abaixo receberá a mesma resolução da imagem que entra nesta camada. A função de ativação é do tipo ReLU. Como esta é a primeira camada, é preciso definir o formato da entrada com `input_shape`:

        keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=[28, 28, 1])

- A seguir adicionamos mais uma camada convolucional. A única diferença agora é que esta contém 64 filtros, também de tamanho 3 por 3.

        keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu'),

- Em seguida, temos uma camada de _max pooling_ que usa um tamanho de pool de 2. Portanto, esta camada divide cada dimensão espacial por um fator de 2.

        keras.layers.MaxPool2D(pool_size=2)

Para imagens maiores, poderíamos repetir essa estrutura de camadas de convolução e camadas de agrupamento várias vezes, até reduzir significativamente o tamanho da imagem. Assim, o número de repetições é um hiperparâmetro da arquitetura da rede.

Observe que o número de filtros tipicamente aumenta à medida em que caminhamos em uma CNN em direção à camada de saída. Isso faz sentido porque o número de detalhes menores de uma imagem costuma ser pequeno (por exemplo, pequenos círculos ou linhas horizontais). Entretanto, existem muitas maneiras diferentes de combiná-los em _features_ de nível superior.

Então é uma prática comum **dobrar** o número de filtros a cada camada de convolução. Uma vez que uma camada de agrupamento divide cada dimensão espacial por um fator de 2, podemos dobrar o número de mapas de _features_ na próxima camada sem medo de explodir o número de parâmetros, o uso de memória ou a carga computacional.

- A seguir, a sequência de camadas deixa de ser convolucional e passa a ser **completamente conectada**, como nas redes neurais tradicionais. A transição é feita por uma camada simples do tipo `Flatten`, que simplesmente enfileira os neurônios ao longo de uma só dimensão.

        keras.layers.Flatten()

- Então adicionamos uma camada de _dropout_, com uma taxa de abandono de 25% cada, que ajuda a reduzir o efeito de _overfitting_. Ou seja, pode ser entendida como uma **camada de regularização**:

        keras.layers.Dropout(0.25)

Uma camada de _dropout_ define aleatoriamente algumas unidades de entrada como 0, segundo uma probabilidade especificada. Isso ocorre somente durante o treinamento, mas não acontece durante a inferência. As entradas mantidas são também ajustadas, de forma que a soma de todas as entradas permaneça a mesma. O uso de _dropout_ tende a desacelerar a convergência, mas geralmente resulta em um modelo muito melhor quando ajustado corretamente.

- Então uma segunda camada `Dense` é adicionada, com 128 neurônios e função de ativação também ReLU:

        keras.layers.Dense(128, activation='relu')

- Uma segunda camada de _dropout_ é colocada, agora com taxa de abandono de 50%:

        keras.layers.Dropout(0.5)

- Finalmente uma camada de saída é adicionada. Aqui o tipo também é `Dense`, mas a função de ativação é trocada para `softmax` para produzir a saída de classificador (uma vez que as 10 classes são mutuamente exclusivas):

        keras.layers.Dense(10, activation="softmax")


In [ ]:
# especificação do modelo
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=[28, 28, 1]),
    keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# resumo legível da arquitetura deste modelo
model.summary()

In [ ]:
# figura da arquitetura deste modelo
keras.utils.plot_model(model, 'model.png', show_shapes=True)

## Compilando a rede neural

Depois que um modelo é criado, é preciso chamar o método `compile()`, especificando a **função de perda** (aqui, a função `sparse_categorical_crossentropy`) e o **otimizador** a ser usado (`nadam`, adequado para redes convolucionais).

Opcionalmente, podemos também pode especificar uma lista de **medidas de desempenho** extras para calcular durante o treinamento e avaliação. Neste caso iremos usar apenas a acurácia com `accuracy`.

In [ ]:
# compilação do modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

## Treinando a rede neural

Para treinar o modelo basta chamar o método `fit()`. Três parâmetros são obrigatórios: as _features_ de treinamento, os rótulos de treinamento e o número de épocas.

Pode ser passado também um conjunto de validação. A biblioteca Keras medirá a perda e as métricas extras ao final de cada época, o que é muito útil para ver como o modelo realmente funciona: se o desempenho no conjunto de treinamento é muito melhor do que no conjunto de validação, provavelmente está ocorrendo _overfitting_.

Como esta rede tem **1.625.866 parâmetros**, é fundamental que este _notebook_ esteja sendo acelerado por uma GPU.

No caso do Colaboratory, o tempo de treinamento com apenas 10 épocas seria de **35 minutos** usando uma CPU. Já com uma GPU cai para cerca de **3 minutos**. Note que a opção TPU do Colaboratory provavelmente só está disponível com o serviço pago, e caso escolhida no serviço gratuito funciona tal como uma CPU.

In [ ]:
# treinamento
%time history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [ ]:
# visualização da evolução das métricas ao longo do treinamento
pd.DataFrame(history.history).plot(figsize=(12, 6))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

## Avaliação da precisão da rede neural

Uma vez treinada, basta avaliarmos sua precisão usando o conjunto de testes. Como pode ser visto abaixo, a acurácia alcançada é de **92,46%**.

É interessante recordar que com um classificador baseado em uma rede neural densa tradicional, com 266.610 neurônios obtivemos uma acurácia de **88,14%**, usando exatamente os mesmos conjuntos de treino, validação e de teste. Naquele momento usamos 25 épocas de treinamento.

In [ ]:
# avaliação usando o conjunto de teste
model.evaluate(X_test, y_test)

# Usando modelos pré-treinados com a Keras

A seguir mostramos um breve exemplo de como podemos utilizar a arquitetura de rede bem mais complexa, a **ResNet50**, para fazer classificação de imagens.

A biblioteca Keras não apenas já tem muitas arquiteturas robustas implementadas, como também permite carregar todos os **pesos** de um modelo de alta qualidade já treinado. Aqui vamos carregar os pesos do treinamento para o _dataset_ ImageNet, com 1000 classes.


In [ ]:
# criação do modelo
model = keras.applications.resnet50.ResNet50(weights='imagenet')

Esse modelo exige que as imagens tenham exatamente a resolução de 224 por 224 _pixels_, então é preciso redimensionar as imagens para este tamanho.

Vamos ilustrar três procedimentos simples: escalonamento, preenchimento de espaço adicional e recorte.

In [ ]:
# escalonamento
images_resized = tf.image.resize(images, [224, 224])
plot_color_image(images_resized[0])
plt.show()

In [ ]:
# preenchimento de espaço adicional
images_resized = tf.image.resize_with_pad(images, 224, 224, antialias=True)
plot_color_image(images_resized[0])
plt.show()

In [ ]:
# recorte
images_resized = tf.image.resize_with_crop_or_pad(images, 224, 224)
plot_color_image(images_resized[0])
plt.show()

In [ ]:
# idealmente o recorte deveria selecionar a área mais importante manualmente
china_box = [0, 0.03, 1, 0.68]
flower_box = [0.19, 0.26, 0.86, 0.7]
images_resized = tf.image.crop_and_resize(images, [china_box, flower_box], [0, 1], [224, 224])
plot_color_image(images_resized[0])
plt.show()
plot_color_image(images_resized[1])
plt.show()

In [ ]:
# cálculo das probabilidades do classificador
inputs = keras.applications.resnet50.preprocess_input(images_resized * 255)
Y_proba = model.predict(inputs)

In [ ]:
# probabilidade das classes
Y_proba.shape

In [ ]:
# decodificador
top_k = keras.applications.resnet50.decode_predictions(Y_proba, top=5)

In [ ]:
# exibição das classes com maior probabilidade
for image_index in range(len(images)):
    print('imagem #{}'.format(image_index))
    for class_id, name, y_proba in top_k[image_index]:
        print('  {} - {:12s} {:.2f}%'.format(class_id, name, y_proba * 100))
    print()

In [ ]:
# resumo legível da arquitetura ResNet50
model.summary()

In [ ]:
# figura da arquitetura ResNet50
keras.utils.plot_model(model, 'ResNet50.png', show_shapes=True)